TODO: This file is supposed to make the plots for the guard coverage by our datasets

In [ ]:
import matplotlib.pyplot as plt
import joblib

In [ ]:
SESSIONS_FILE_FILE = '../sumo_pipeline/session_correlation/results/data/results_by_eu_country_small_OSTest.pickle'

guard_nodes = joblib.load(SESSIONS_FILE_FILE)

### Plot guard probability per AS and country

In [ ]:
ases_count = {}
ases_countries = {}
country_codes_count = {}

In [ ]:
def count_occurences():
    total_iterations = 0
    
    for session in guard_nodes:
        # client side
        if session.client_as not in ases_count:
            ases_count[session.client_as] = 0
        ases_count[session.client_as] += 1

        if session.client_country_code not in country_codes_count:
            country_codes_count[session.client_country_code] = 0
        country_codes_count[session.client_country_code] += 1

        if session.client_as not in ases_countries:
            ases_countries[session.client_as] = set()
        ases_countries[session.client_as].add(session.client_country_code)

        # onion service side
        if session.onion_as not in ases_count:
            ases_count[session.onion_as] = 0
        ases_count[session.onion_as] += 1

        if session.onion_country_code not in country_codes_count:
            country_codes_count[session.onion_country_code] = 0
        country_codes_count[session.onion_country_code] += 1

        if session.onion_as not in ases_countries:
            ases_countries[session.onion_as] = set()
        ases_countries[session.onion_as].add(session.onion_country_code)

        total_iterations += 1

    return total_iterations

total_iterations = count_occurences()

print("\nisps_count", isps_count)
print("\nases_count", ases_count)
print("\ncountries_count", country_codes_count)
print("\nisps_countries", isps_countries)
print("\nases_countries", ases_countries)
print("\ntotal_iterations", total_iterations)

In [ ]:
as_guard_probabilities = {}

def plot_guard_probability_per_as():
    plt.figure(figsize=(8, 6))  # Width: 8 inches, Height: 6 inches
    
    n_els_to_show = 20

    # Extract keys and lengths
    keys = list(ases_count.keys())
    total = sum(ases_count.values())
    lengths = [(s/total) for s in ases_count.values()]

    for key, prob in zip(keys, lengths):
        as_guard_probabilities[key] = prob

    # Combine keys and lengths using zip
    combined = list(zip(keys, lengths))
    # Sort the combined list based on lengths in descending order
    combined.sort(key=lambda x: x[1], reverse=True)
    # Unzip the sorted list back into separate lists
    sorted_keys, sorted_lengths = zip(*combined)

    # Create a bar plot
    plt.bar(sorted_keys[:n_els_to_show], sorted_lengths[:n_els_to_show])

    plt.xticks(rotation=90)

    # Add labels and title
    plt.xlabel(f'Top {n_els_to_show} ASes')
    plt.ylabel('Guard probability')

    # Display the plot
    plt.show()

plot_guard_probability_per_as()

In [ ]:
top_country_guard_probabilities = {}

def plot_guard_probability_per_country():
    plt.figure(figsize=(8, 6))  # Width: 8 inches, Height: 6 inches
    
    n_els_to_show = 20

    # Extract keys and lengths
    keys = list(country_codes_count.keys())
    total = sum(country_codes_count.values())
    lengths = [(s/total) for s in country_codes_count.values()]

    # Combine keys and lengths using zip
    combined = list(zip(keys, lengths))
    # Sort the combined list based on lengths in descending order
    combined.sort(key=lambda x: x[1], reverse=True)
    # Unzip the sorted list back into separate lists
    sorted_keys, sorted_lengths = zip(*combined)

    # Create a bar plot
    plt.bar(sorted_keys[:n_els_to_show], sorted_lengths[:n_els_to_show])

    for key, value in zip(sorted_keys[:n_els_to_show], sorted_lengths[:n_els_to_show]):
        top_country_guard_probabilities[key] = value

    plt.xticks(rotation=90)

    # Add labels and title
    plt.xlabel(f'Top {n_els_to_show} countries')
    plt.ylabel('Guard probability')

    # Display the plot
    plt.show()

plot_guard_probability_per_country()

### Plot probability of guard being under same AS and country

In [ ]:
def count_matching_countries():
    total_matching_countries = 0
    matching_countries = {}
    
    for session in data:
        if session.client_country_code == session.onion_country_code:
            if session.client_country_code not in matching_countries:
                matching_countries[session.client_country_code] = 0
            matching_countries[session.client_country_code] += 1
            total_matching_countries += 1

    return total_matching_countries, matching_countries

total_matching_countries, matching_countries = count_matching_countries()
print("\ntotal_sessions_made", len(guard_nodes))
print("\ntotal_matching_countries", total_matching_countries)
print("\ntotal_occurences", len(data))
print("\ntotal_matching_countries / total_occurences", total_matching_countries / len(data))
print("\nmatching_countries", matching_countries)

In [ ]:
def count_matching_ases():
    total_matching_ases = 0
    matching_ases = {}
    
    for session in data:
        if session.client_as == session.onion_as:
            if session.client_as not in matching_ases:
                matching_ases[session.client_as] = 0
            matching_ases[session.client_as] += 1
            total_matching_ases += 1
    
    return total_matching_ases, matching_ases

total_matching_ases, matching_ases = count_matching_ases()
print("\ntotal_matching_ases", total_matching_ases)
print("\ntotal_occurences", len(data))
print("\ntotal_matching_ases / total_occurences", total_matching_ases / len(data))
print("\nmatching_ases", matching_ases)

In [ ]:
def plot_matching_countries():
    plt.figure(figsize=(8, 6))  # Width: 8 inches, Height: 6 inches

    # Extract keys and lengths
    keys = list(matching_countries.keys())
    lengths = [(s/len(data)) for s in matching_countries.values()]

    # Combine keys and lengths using zip
    combined = list(zip(keys, lengths))
    # Sort the combined list based on lengths in descending order
    combined.sort(key=lambda x: x[1], reverse=True)
    # Unzip the sorted list back into separate lists
    sorted_keys, sorted_lengths = zip(*combined)

    # Create a bar plot
    plt.bar(sorted_keys, sorted_lengths)

    plt.xticks(rotation=90)

    # Add labels and title
    plt.xlabel(f'Top countries')
    plt.ylabel('Probability of client\'s guard and onion\'s guard being in the same country')

    # Display the plot
    plt.show()

plot_matching_countries()

In [ ]:
def get_as_number(as_name):
    # Define a regex pattern to match common AS number formats
    pattern = r'\bAS\s?(\d+)\b'
    
    # Search for the pattern in the AS name
    match = re.search(pattern, as_name, re.IGNORECASE)
    
    if match:
        return match.group()
        #return match.group(1)
    else:
        return None

In [ ]:
def plot_matching_ases():
    plt.figure(figsize=(8, 6))  # Width: 8 inches, Height: 6 inches

    # Extract keys and lengths
    keys = []
    for as_name in matching_ases.keys():
        keys.append(get_as_number(as_name))
    lengths = [(s/len(data)) for s in matching_ases.values()]

    # Combine keys and lengths using zip
    combined = list(zip(keys, lengths))
    # Sort the combined list based on lengths in descending order
    combined.sort(key=lambda x: x[1], reverse=True)
    # Unzip the sorted list back into separate lists
    sorted_keys, sorted_lengths = zip(*combined)

    # Create a bar plot
    plt.bar(sorted_keys, sorted_lengths)

    plt.xticks(rotation=90)

    # Add labels and title
    plt.xlabel(f'Top ASes')
    plt.ylabel('Probability of client\'s guard and onion\'s guard being in the same AS')

    # Display the plot
    plt.show()

plot_matching_ases()